# Laplace

Notebook to perform Laplace transformations

Author: Lucas Schneider

---

## Initialization

In [1]:
import sympy as sym
import numpy as np

from sympy.integrals import laplace_transform
from sympy.integrals import inverse_laplace_transform

from IPython.display import display
from IPython.display import Math
from sympy.interactive import printing

# sym.init_printing()

In [2]:
t_var = sym.symbols('t', real=True)
s_var = sym.symbols('s')

## Direct Transform

In [ ]:
# Other symbols declaration
phi = sym.symbols('φ', real=True)

In [ ]:
# Input
expression = sym.cos(t_var)

expression *= sym.Heaviside(t_var)
print(str(expression).replace('**','^'))
expression

In [ ]:
U = laplace_transform(expression, t_var, s_var)
U[0]

## Inverse Transform

In [3]:
def poly_from_list(num_list, den_list, print_latex=True, print_string=False):
    '''
    Transforms the list of coefficients into sympy polynomial objects

    Parameters
    ----------
    num_list : list of floats
    den_list : list of floats
    print_latex : bool
    print_string : bool

    Returns
    -------
    num_pol : sympy.Poly
    den_pol : sympy.Poly
    
    '''

    num_pol = sym.Poly(num_list, s_var)
    den_pol = sym.Poly(den_list, s_var)

    F = num_pol / den_pol
    F_str = str(F).replace('**','^').replace('I', 'i').replace('exp', 'e^')
    
    if print_latex:
        display(Math(f'F(s) = {printing.default_latex(F)}'))

    if print_string:
        print(f'String for Wolfram: {F_str}\n')

    return num_pol, den_pol

In [4]:
def partial_fractions(num_pol, den_pol, print_latex=True, print_string=False):
    '''
    Apply the partial fraction transformation with the rational polynomial

    Parameters
    ----------
    num_pol : sympy.Poly
    den_pol : sympy.Poly
    print_latex : bool
    print_string : bool

    Returns
    -------
    complete_terms : list of sympy.Expr
    fraction_terms : list of sympy.Expr
    '''

    sum_of_complete, sum_of_fraction = sym.div(num_pol, den_pol)

    sum_of_fraction = sym.apart(sum_of_fraction / den_pol, full=True).doit()

    fraction_terms = list(sum_of_fraction.args)
    fraction_terms = [frac.simplify() for frac in fraction_terms]

    complete_terms = [list(reversed(sum_of_complete.coeffs()))[i] * s_var ** i for i in range(len(sum_of_complete.coeffs()))]

    F = sum(complete_terms) + sum(fraction_terms)
    F_str = str(F).replace('**','^').replace('I', 'i').replace('exp', 'e^')

    if print_latex:
        display(Math(f'F(s) = {printing.default_latex(F)}'))

    if print_string:
        print(f'String for Wolfram: {F_str}\n')

    return complete_terms, fraction_terms


In [5]:
def residue(num_pol, den_pol, print_latex=True, print_string=False):
    '''
    Calulates the partial fraction and residues for the given rational polynomial

    Parameters
    ----------
    num_pol : sympy.Poly
    den_pol : sympy.Poly
    print_latex : bool
    print_string : bool

    Returns
    -------
    residues: list of floats
        Resiudes of the fraction part
    poles: list of floats
        Poles of the fraction part
    multi: list of floats
        Poles's multiplicity
    complete : list of floats
        Coefficients of complete part
    '''
    residues = []
    poles = []
    multi = []
    complete = []

    complete_terms, fraction_terms = partial_fractions(num_pol, den_pol, print_latex, print_string)

    if len(complete_terms) > 1:
        complete = sym.Poly(sum(complete_terms)).coeffs()
    elif len(complete_terms) == 1:
        complete = complete_terms

    roots_dict = sym.polys.polyroots.roots(den_pol)
    for root in roots_dict.keys():
        for i in range(roots_dict[root]):
            poles.append(root)
            multi.append(i+1)

    for i in range(len(poles)):
        for frac in fraction_terms:
            possible_residue = (frac * ((s_var - poles[i]) ** multi[i])).simplify()
            if str(s_var) not in str(possible_residue):
                # print(possible_residue, poles[i], sep='   ')
                residues.append(possible_residue)

    # TODO: Print residues table (with Pandas?)

    return residues, poles, multi, complete

In [6]:
def conjugate_already_seen(pole, multi, poles_seen):
    '''
    Checks if an specific pole or its conjugate has already been computed before, with the same multiplicity

    Parameters
    ----------
    pole : copmlex
    multi : int
    poles_seen : list of tuples, with pole and its multiplicity

    Returns
    -------
    True or False
    '''
    for pole_seen, multi_seen in poles_seen:
        if (pole.equals(pole_seen) or (pole.conjugate()).equals(pole_seen)) and (multi == multi_seen):
            return True

    return False


def ILP_from_residues(residues, poles, multi, complete, complex_simplify=True, print_latex=True, print_string=False):
    '''
    Perform the Inverse Laplace Transform from the given residues

    Parameters
    ----------
    residues: list of floats
    poles: list of floats
    multi: list of floats
    complete : list of floats

    print_latex : bool
    print_string : bool

    Returns
    -------
    f_terms : list of sympy.Expr
    '''

    f_terms_complete = [complete[i] * sym.diff(sym.DiracDelta(s_var), s_var, len(complete) -1 -i) for i in range(len(complete))]

    f_terms_fraction = []

    complex_poles_pairs = []
    for i in range(len(residues)):
        k_min_1 = multi[i] - 1
        if (not (sym.im(poles[i])).equals(0)) and complex_simplify:
            if not conjugate_already_seen(poles[i], multi[i], complex_poles_pairs):
                if sym.im(residues[i]) < 0:
                    Ak = residues[i].conjugate()
                    pk = poles[i].conjugate()
                else:
                    Ak = residues[i]
                    pk = poles[i]

                term = sym.Mul(t_var ** (k_min_1) / sym.factorial(k_min_1), 2 * sym.sqrt(sym.re(Ak) ** 2 + sym.im(Ak) ** 2), sym.exp(sym.re(pk) * t_var), sym.cos(sym.Add(sym.im(pk)*t_var, sym.arg(Ak), evaluate=False)))

                complex_poles_pairs.append((poles[i], multi[i]))
            else:
                continue
        else:
            term = sym.Mul(residues[i] / sym.factorial(k_min_1), t_var ** (k_min_1), sym.exp(poles[i] * t_var), evaluate=False)
        
        # print(term)
        # if multi[i] == 2:
        f_terms_fraction.append(term)

    f = sum(f_terms_complete) + sym.Mul(sum(f_terms_fraction), sym.Heaviside(t_var), evaluate=False)
    f_str = str(f).replace('**','^').replace('I', 'i').replace('exp', 'e^')

    if print_latex:
        display(Math(f'f(t) = {printing.default_latex(f)}'))

    if print_string:
        print(f'String for Wolfram: {f_str}\n')

    return f


In [7]:
N_list = [1, 5, 4, 3, 1]
D_list = [1, 3, 2, 0]

pol = poly_from_list(N_list, D_list)
res = residue(*pol)
f = ILP_from_residues(*res, print_string=True)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

String for Wolfram: (1/2 + 2*e^(-t) - 13*e^(-2*t)/2)*Heaviside(t) + 2*DiracDelta(s) + DiracDelta(s, 1)



In [8]:
# Apply the Inverse Laplace Transform from sympy
f_complete_terms = []
f_fraction_terms = []

complete_terms, fraction_terms = partial_fractions(*pol)

for term in complete_terms:
    transform = inverse_laplace_transform(term, s_var, t_var)
    f_complete_terms.append(transform)

for term in fraction_terms:
    transform = inverse_laplace_transform(term, s_var, t_var)
    f_fraction_terms.append(transform)

f = sum(f_complete_terms) + sum(f_fraction_terms)
print(str(f).replace('**','^'))
f

<IPython.core.display.Math object>

InverseLaplaceTransform(s, s, t, _None) + 2*DiracDelta(t) + Heaviside(t)/2 + 2*exp(-t)*Heaviside(t) - 13*exp(-2*t)*Heaviside(t)/2


InverseLaplaceTransform(s, s, t, _None) + 2*DiracDelta(t) + Heaviside(t)/2 + 2*exp(-t)*Heaviside(t) - 13*exp(-2*t)*Heaviside(t)/2

In [ ]:
f.evalf(subs={t_var: 0.5})